<a href="https://colab.research.google.com/github/abellizzistudent/CST383_Project/blob/main/SAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyreadstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
#import pyreadstat
from matplotlib import rcParams

# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html
sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7

In [4]:
df = pd.read_spss('/content/ATPW45.sav')
df.tail()

PyreadstatError: ignored


# -*- coding: utf-8 -*-
"""
Created on Sun Jan 29 12:45:03 2023

@author: Anna
"""
#import pyreadstat
import pandas as pd

#Method One:
#df, meta = pyreadstat.read_sav('C:/Users/Anna/Downloads/W45_Feb19/W45_Feb19/ATP W45.sav')
#CHECKS:
#print(type(df))
#print(df.head())

#Method Two:
df = pd.read_spss('C:/Users/Anna/Desktop/CST383-Data Science/W4/W87_Apr21/ATP W87.sav')
#CHECKS:
#print(df.tail())

# Uncomment to print list of column names:
#for col in df.columns:
#    print(col)

#print(df["GUNSTRICT_W87"].shape)

#Check race/sentiment demographics
#race_gun = df[["F_RACECMB", "GUNSTRICT_W87"]]
#CHECK:
#print (race_gun.head())
#print(df.value_counts('GUNSTRICT_W87'))

#Aggregate sentiment by race:
df['COUNTER'] =1       #initially, set that counter to 1.
group_data = df.groupby(['F_RACECMB', 'GUNSTRICT_W87'])['COUNTER'].sum() #sum function
print("\n Gun law sentiment by race: \n")
print(group_data)

df['COUNTER'] =1       #initially, set that counter to 1.
group_data = df.groupby(['F_RACECMB', 'GUNPRIORITY1_b_W87'])['COUNTER'].sum() #sum function

print("\n Preventing people with mental illnesses from purchasing guns: \n")
print(group_data)

df['COUNTER'] =1       #initially, set that counter to 1.
group_data = df.groupby(['F_RACECMB', 'GUNPRIORITY1_c_W87'])['COUNTER'].sum() #sum function

print("\n Banning assault-style weapons: \n")
print(group_data)

df['COUNTER'] =1       #initially, set that counter to 1.
group_data = df.groupby(['F_RACECMB', 'GUNPRIORITY1_f_W87'])['COUNTER'].sum() #sum function

print("\n Making private gun sales and sales at gun shows subject to background checks: \n")
print(group_data)


df['COUNTER'] =1       #initially, set that counter to 1.
group_data = df.groupby(['F_RACECMB', 'GUNPRIORITY2_j_W87'])['COUNTER'].sum() #sum function

print("\n Shortening waiting periods for people who want to buy guns legally: \n")
print(group_data)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# San Francsico Bite Data 2014-2018

San Francisco Bite Data, 2014-2018 - Total rows: 3863
Exported PDF to Exel via Adobe Acrobat
Removed rows that contained only total by breed sections - 267 rows
Removed header rows that appeard within the doc as a PDF - 475 rows
Split bite severity into two seperate columns - BITE_CODE and BITE_SEVERITY
***Retaining both columns for now, both are not necessary since they contain the same infomation
Replaced alpha "U" code in BITE_SEVERITY for UNKNOWN to 9 to match numeric convention of hte column
Removed spaces in BITE_SEVERITY alpha codes for ease of search
Removed BITE column because each row value for that column was 1. Each row represents a single event already, column is not necessary.
Total Rows: 3626
Total Cols: 5
Change column names to all lowercase
Imported as CSV - sf-raw-data-dog-bites-2014-2018-CLEAN.csv


# Import SF data

In [7]:
df_sf = pd.read_csv("/content/sf-raw-data-dog-bites-2014-2018-CLEAN.csv")
df_sf.head()
###breed_group may be an unnecessary column:
#breed_group = df_sf.breed_group
#print(breed_group.unique().size)
#primary_breed = df_sf.primary_breed
#print(primary_breed.unique().size)




,breed_group,primary_breed,bite_code,bite_severity,gender
0,AFFENPINSCHER,AFFENPINSCHER,0,SNGLNIP,S
1,AFFENPINSCHER,AFFENPINSCHER,1,SNGLB,U
2,AFGHAN HOUND,AFGHAN HOUND,1,SNGLB,N
3,AIREDALE TERR,AIREDALE TERR,0,SNGLNIP,N
4,AIREDALE TERR,AIREDALE TERR,1,SNGLB,N


Removed columns Borough, ZipCode, Species
Combined Gender and Spay/Neuter columns to lmatch format of DF_SF gender column
Removed Key column and Date of bite column
Ages modified to standard values, removed alphas and turned numbers into decimal value in years.
Added Bad_Data column to tag potentially bad rows that could be thrown out of the dataset.
Added breed_group column to match df_sf dataset and clarify primary_breed values that specify a group.
added Multi-Dog column for entries invloving more than one dog in description. Low numbers, may consider dropping these rows.

Cleanup is not complete, need to standardize breed column and breed_group column

The data does not include whether one dog is a multi-offender. 



In [11]:
df_ny = pd.read_csv("/content/ny-dog-bites-2015-2021-CLEAN.csv")
df_ny.head()

,breed_group,primary_breed,age,modified_age,gender,multi_dog,bad_data
0,UNKNOWN,UNKNOWN,U,U,U,F,T
1,UNKNOWN,UNKNOWN,U,U,U,F,T
2,NaN,Pit Bull,U,U,U,F,F
3,NaN,Mixed/Other,4,4,M,F,F
4,NaN,Pit Bull,U,U,U,F,F
